<a href="https://colab.research.google.com/github/pallav0007/Custom_Object_Detection/blob/main/Custom_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#mounting the drive 
import os
from google.colab import drive
MOUNTPOINT = '/content/gdrive'
DATADIR = os.path.join(MOUNTPOINT, 'MyDrive')
drive.mount(MOUNTPOINT)

In [ ]:
#download the web file for the pretrained model 
!wget https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/pretrained-yolov3.h5

In [ ]:
#optionally move the temporary file to mounted driv
!mv /content/models/pretrained-yolov3.h5 /content/gdrive/My\ Drive/

In [ ]:
#install the dependencies for the project
!pip install keras==2.4.3 numpy==1.19.3 pillow==7.0.0 scipy==1.4.1 h5py==2.10.0 matplotlib==3.3.2 opencv-python keras-resnet==0.2.0
!pip install imageai

In [ ]:
#intializing the trainer object with dataset and labels
#we have set model type to YOLOv3

import imageai
from imageai.Detection.Custom import DetectionModelTrainer

dataset=os.path.join(MOUNTPOINT, 'MyDrive','dataset')
type_of_objects=["thumbsup","thumbsdown","bulb","bottle","face","curtain"]
trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory=dataset)
trainer.setTrainConfig(object_names_array=type_of_objects, batch_size=4, num_experiments=10, train_from_pretrained_model="pretrained-yolov3.h5")
trainer.trainModel()

In [ ]:
#colab code to access webcam
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
%matplotlib inline
#The line above is necesary to show Matplotlib's plots inside a Jupyter Notebook

import cv2
from matplotlib import pyplot as plt

#function to display detected box points in image
def display_detection(name,percentage_probability,box_points):
  x,y,w,h=box_points
  image = cv2.imread("/content/holo3-detected.jpg")
  image = cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 1)
  cv2.putText(image,f"{name}-{percentage_probability}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
  #Show the image with matplotlib
  plt.imshow(image)
  plt.show()

In [ ]:
#initialize the detection object 
#set model path to model saved by the trainer object
#use configuration save by trainer in config.json
#use the image captured by webcam (by defaulted name photo.jpg)

from imageai.Detection.Custom import CustomObjectDetection

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath("/content/gdrive/MyDrive/dataset/models/detection_model-ex-006--loss-0041.969.h5") 
detector.setJsonPath("/content/gdrive/MyDrive/dataset/json/detection_config.json")
detector.loadModel()
detections = detector.detectObjectsFromImage(input_image="photo.jpg", output_image_path="detected.jpg")
for detection in detections:
    display_detection(**detection)
    print(detection["name"], " : ", detection["percentage_probability"], " : ", detection["box_points"])